# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 43.301 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(10*1000*1000)]

In [2] used 309.605 MiB RAM in 1.438s, peaked 0.000 MiB above current, total RAM usage 352.906 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

10 loops, best of 3: 42.9 ms per loop
In [3] used 0.398 MiB RAM in 1.820s, peaked 0.000 MiB above current, total RAM usage 353.305 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((len(a) / t.best / 1e6), 1))

MFLOPS: 233.1
In [4] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 353.305 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## NumPy

In [5]:
import numpy as np

In [5] used 13.773 MiB RAM in 0.088s, peaked 0.000 MiB above current, total RAM usage 367.078 MiB


In [6]:
na = np.array(a, dtype=np.float64)

In [6] used 76.418 MiB RAM in 0.199s, peaked 0.000 MiB above current, total RAM usage 443.496 MiB


In [7]:
print("SIZE:", round((na.size * na.itemsize) / 2**20., 3))

SIZE: 76.294
In [7] used 0.020 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


We see that, with 8 bytes/element, NumPy is a very efficient container.

In [8]:
t = %timeit -o sum(na)

1 loop, best of 3: 705 ms per loop
In [8] used 0.000 MiB RAM in 2.828s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


In [9]:
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

MFLOPS: 14.177
In [9] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


### Exercise

The performance for NumPy is several times slower than the computation with the list.  Why so?

*Hint: * We are using sum() which is a Python function.

### Solution

NumPy has a lot of overhead in producing a Python integer for every element in the array for feeding it to the sum().

*Hint:* Use internal NumPy methods (ufuncs) when possible.

In [10]:
t = %timeit -o na.sum()

100 loops, best of 3: 3.45 ms per loop
In [10] used 0.000 MiB RAM in 1.426s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


In [11]:
print("FLOPS:", round(len(a) / t.best / 1e6, 3))

FLOPS: 2897.656
In [11] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


This is more than 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [12]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

21.589217096585486

In [12] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 443.516 MiB


So, in this case the memory bandwidth is ~ 17 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [13]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'lz4hc'
bcolz.defaults.cparams['clevel'] = 3
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.0.0
NumPy version:     1.11.1
Blosc version:     1.8.1 ($Date:: 2016-04-08 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.6.1
Python version:    3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux-x86_64
Byte-ordering:     little
Detected cores:    8
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


8

In [13] used 30.719 MiB RAM in 0.263s, peaked 0.000 MiB above current, total RAM usage 474.234 MiB


In [14]:
ca = bcolz.carray(na)

In [14] used 2.215 MiB RAM in 0.045s, peaked 0.000 MiB above current, total RAM usage 476.449 MiB


In [15]:
print("mem_used:", mw.measurements.memory_delta)

mem_used: 2.21484375
In [15] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 476.449 MiB


Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [16]:
ca

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 1.42 MB; ratio: 53.80
  cparams := cparams(clevel=3, shuffle=1, cname='lz4hc')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [16] used 0.707 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 477.156 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [17]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 18.1 ms per loop
MFLOPS: 553.612
In [17] used 0.000 MiB RAM in 7.578s, peaked 0.000 MiB above current, total RAM usage 477.156 MiB


This is around 2~5x slower (depending on the machine) than a regular NumPy array, but the size of the array is a quite impressive 50x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [18]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [18] used 76.648 MiB RAM in 0.261s, peaked 0.000 MiB above current, total RAM usage 553.805 MiB


In [19]:
cau

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 76.52 MB; ratio: 1.00
  cparams := cparams(clevel=0, shuffle=1, cname='lz4hc')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [19] used 0.023 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 553.828 MiB


In [20]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 8.88 ms per loop
MFLOPS: 1126.118
In [20] used 0.000 MiB RAM in 3.707s, peaked 0.000 MiB above current, total RAM usage 553.828 MiB


As we can see, the times with an uncompressed `carray` are noticieably faster than with a compressed one, so compressing is not the only source of the overhead.  The other source of the difference is the memory layout of the different containers (bcolz's carray data container layout is a bit more complex than NumPy).

So, while bcolz allows to use compressed in-memory data containers, this usually represents more cost in performance (compared with NumPy).  But sometimes you may prefer to keep more data in-memory and assume that the computations are going to be slower.

## Exercise

bcolz uses Blosc, a multithreaded meta-compressor, to do the compression under the hood.  Blosc can use different codecs, and each one has different behavior in terms of performance.  Given the next computation:

In [21]:
bcolz.defaults.cparams['cname'] = 'blosclz'   # try also with 'lz4', 'lz4hc', 'zlib'
bcolz.defaults.cparams['clevel'] = 9          # try from 1 to 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE   # try with bcolz.NOSHUFFLE and bcolz.BITSHUFFLE too
bcolz.set_nthreads(8)
ca = bcolz.carray(na)
%timeit ca.sum()
print("carray sizes--> uncompr: %.3f MB, compr: %.3f MB, ratio: %.3f" % (
    ca.nbytes / 2.**20, ca.cbytes/ 2**20., (ca.nbytes / ca.cbytes)))

10 loops, best of 3: 21.9 ms per loop
carray sizes--> uncompr: 76.294 MB, compr: 1.013 MB, ratio: 75.314
In [21] used 0.801 MiB RAM in 0.932s, peaked 0.000 MiB above current, total RAM usage 554.629 MiB


Play with the different parameters and see:

1) Which provides the best compression

2) Which the fastest speed

3) The combination that strikes a good balance between compression and performance